In [ ]:
#import tensorflow as tf
#print(tf.__version__)
#print(tf.test.is_gpu_available())

In [ ]:
import numpy as np

from keras.models import Model
from keras.layers import Input, LSTM, Dense

print("DONE")

In [ ]:
with open('../input/largetextfiles/world192small.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()

with open('../input/largetextfiles/world192small_baptista.txt', 'r', encoding='utf-8') as f:
    text_encrypted = f.readlines()

print("DONE")

In [ ]:
print(len(text))
print(len(text_encrypted))

In [ ]:
# to store the samples
en_samples = []
de_samples = []

# to store the characters
en_chars = set()
de_chars = set()

# TODO.. cleanup.. maybe merge plain and encrypted beforehand
# sanity check for next fori
assert len(text) == len(text_encrypted)

# split the samples and get the character sets
for i in range(len(text)):
    en_ = text_encrypted[i]
    de_ = text[i]
    de_ = '\t' + de_
    for char in de_:
        if char not in de_chars:
            de_chars.add(char)
    for char in en_:
        if char not in en_chars:
            en_chars.add(char)
    en_samples.append(en_)
    de_samples.append(de_)

de_chars.add('\n')
de_chars.add('\t')

en_chars.add('\n')
en_chars.add('\t')

print("DONE")

In [ ]:
# chars <-> ints convert

de_char_to_int = dict()
de_int_to_char = dict()
en_char_to_int = dict()
en_int_to_char = dict()

for i, char in enumerate(de_chars):
    de_char_to_int[char] = i
    de_int_to_char[i] = char
    
for i, char in enumerate(en_chars):    
    en_char_to_int[char] = i
    en_int_to_char[i] = char

print("DONE")

In [ ]:
# get lengths and sizes
num_en_chars = len(en_chars)
num_de_chars = len(de_chars)

max_en_chars_per_sample = max([len(sample) for sample in en_samples])
max_de_chars_per_sample = max([len(sample) for sample in de_samples])

num_en_samples = len(en_samples)
num_de_samples = len(de_samples)

print("DONE")

In [ ]:
# initiate numpy arrays to hold the data for the seq2seq model
encoder_input_data = np.zeros((num_en_samples,
                               max_en_chars_per_sample,
                               num_en_chars),
                              dtype='float32')

decoder_input_data = np.zeros((num_de_samples,
                               max_de_chars_per_sample,
                               num_de_chars),
                              dtype='float32')

target_data = np.zeros((num_de_samples,
                       max_de_chars_per_sample,
                       num_de_chars),
                      dtype='float32')

print("DONE")

In [ ]:
# one-hot encode
# takes too much space, thus 1/4 of original file
# TODO.. next step: use embedding, words instead of chars
for i, (en_sample, de_sample) in enumerate(zip(en_samples, de_samples)):
    for char, en_char in enumerate(en_sample):
        encoder_input_data[i, char, en_char_to_int[en_char]] = 1
    for char, de_char in enumerate(de_sample):
        decoder_input_data[i, char, de_char_to_int[de_char]] = 1
        if char > 0 :
            target_data[i, char - 1, de_char_to_int[de_char]]  = 1
            
print("DONE")

In [ ]:
# #################################################################################################
# #################################################################################################
# #################################################################################################

# END TWDS NLP1 P1

# START TWDS NLP1 P2 / NLP2 ?

# #################################################################################################
# #################################################################################################
# #################################################################################################

In [ ]:
# DATA CLEANUP. OR IS IT?.. SHOULD RECONSIDER

import string, re
from string import digits

# Convert text to lowercase :
text_encrypted=list(map(lambda x: x.lower(), text_encrypted))
text=list(map(lambda x: x.lower(), text))

# Process commas :
text_encrypted=list(map(lambda x: re.sub("'", '', x), text_encrypted))
text_encrypted=list(map(lambda x: re.sub(",", ' COMMA', x), text_encrypted))

text=list(map(lambda x: re.sub("'", '', x), text))
text=list(map(lambda x: re.sub(",", ' COMMA', x),text))

# Getting rid of punctuation
exclude = set(string.punctuation)
text_encrypted=list(map(lambda x: ''.join(ch for ch in x if ch not in exclude), text_encrypted))
text=list(map(lambda x: ''.join(ch for ch in x if ch not in exclude), text))

# Getting rid of digits
remove_digits = str.maketrans('', '', digits)
text_encrypted=list(map(lambda x: x.translate(remove_digits), text_encrypted))
text=list(map(lambda x: x.translate(remove_digits), text))

print("DONE")

In [ ]:
# Sample Processing

# Appending SOS andEOS to target data : 
text = list(map(lambda x : 'SOS_ '+ x + ' _EOS', text))

# Create word dictionaries :
en_words=set()
for line in text_encrypted:
    for word in line.split():
        if word not in en_words:
            en_words.add(word)
    
de_words=set()
for line in text:
    for word in line.split():
        if word not in de_words:
            de_words.add(word)
            
# get lengths and sizes :
num_en_words = len(en_words)
num_de_words = len(de_words)

max_en_words_per_sample = max([len(sample.split()) for sample in text_encrypted])+5
max_de_words_per_sample = max([len(sample.split()) for sample in text])+5

num_en_samples = len(text_encrypted)
num_de_samples = len(text)

# Get lists of words :
input_words = sorted(list(en_words))
target_words = sorted(list(de_words))

en_token_to_int = dict()
en_int_to_token = dict()

de_token_to_int = dict()
de_int_to_token = dict()

#Tokenizing the words ( Convert them to numbers ) :
for i,token in enumerate(input_words):
    en_token_to_int[token] = i
    en_int_to_token[i]     = token

for i,token in enumerate(target_words):
    de_token_to_int[token] = i
    de_int_to_token[i]     = token

# initiate numpy arrays to hold the data that our seq2seq model will use:
encoder_input_data = np.zeros(
    (num_en_samples, max_en_words_per_sample),
    dtype='float32')
decoder_input_data = np.zeros(
    (num_de_samples, max_de_words_per_sample),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_de_samples, max_de_words_per_sample, num_de_words),
    dtype='float32')

# Process samples, to get input, output, target data:
for i, (input_text, target_text) in enumerate(zip(text_encrypted, text)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = en_token_to_int[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = de_token_to_int[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, de_token_to_int[word]] = 1.

print("DONE")

In [ ]:
# Importing needed dependencies :
import pandas as pd
import numpy as np
import string
from string import digits

from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint

from keras.layers import CuDNNLSTM

print("DONE")

In [ ]:
# Defining some constants: 
vec_len       = 300   # Length of the vector that we willl get from the embedding layer
latent_dim    = 1024  # Hidden layers dimension 
dropout_rate  = 0.2   # Rate of the dropout layers
batch_size    = 64    # Batch size
epochs        = 10    # Number of epochs

print("DONE")

In [ ]:
# Define an input sequence and process it.
# Input layer of the encoder :
encoder_input = Input(shape=(None,))

# Hidden layers of the encoder :
encoder_embedding = Embedding(input_dim = num_en_words, output_dim = vec_len)(encoder_input)
encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)
encoder_LSTM      = LSTM(latent_dim, return_sequences=True)(encoder_dropout)
#encoder_LSTM      = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_dropout)


# Output layer of the encoder :
#encoder_LSTM2_layer = CuDNNLSTM(latent_dim, return_state=True)
encoder_LSTM2_layer = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM2_layer(encoder_LSTM)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

print("DONE")

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
# Input layer of the decoder :
decoder_input = Input(shape=(None,))

# Hidden layers of the decoder :
decoder_embedding_layer = Embedding(input_dim = num_de_words, output_dim = vec_len)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_dropout_layer = (TimeDistributed(Dropout(rate = dropout_rate)))
decoder_dropout = decoder_dropout_layer(decoder_embedding)

#decoder_LSTM_layer = CuDNNLSTM(latent_dim, return_sequences=True)
decoder_LSTM_layer = LSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

#decoder_LSTM_2_layer = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2,_,_ = decoder_LSTM_2_layer(decoder_LSTM)

# Output layer of the decoder :
decoder_dense = Dense(num_de_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_LSTM_2)

print("DONE")

In [ ]:
model = Model([encoder_input, decoder_input], decoder_outputs)

In [ ]:
model.summary()

In [ ]:
#!pip install --upgrade livelossplot
#from livelossplot import PlotLossesKeras

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#from keras.callbacks import EarlyStopping
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=True,
                    validation_split=0.2,
                    callbacks=callbacks_list,
                    verbose=1
                   )

In [ ]:
model.save('model.h5')